<a href="https://colab.research.google.com/github/sanderjenk/Predicting-PL-Fantasy/blob/main/Fantasy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ML Football

## Data
https://github.com/vaastav/Fantasy-Premier-League

I used gameweek data of all players from the repository mentioned above, by checking out the whole repo. 

## Preprocessing
Most of the colums are not usable by default for training the model, since we don't know beforehand how many assists players score in the next match. But we do know averages up to that gameweek. I made a script that goes to each players subfolder, goes through each row and adds averages up to that gameweek as new columns, and in the end creates a one csv file that contains every player's data. 

### Script:

```
import glob
import pandas as pd

#columns that are not useful
drop_list = ["element", "fixture", "kickoff_time", "round", "selected", "transfers_in", "transfers_out", "transfers_balance"]

# list of columns that will not have an average column
ignore_avg_list = ["was_home"]

# list of columns that will not be dropped after the average column is added
ignore_drop = ["total_points"]

# path of the player data folders
files = glob.glob("C:\\Users\\sande\\Desktop\\Kool\\courses\\machine learning\\Fantasy-Premier-League\\data\\2019-20\\players\\*\\gw.csv")
result = []
for f in files:
    df = pd.read_csv(f)
    df = df.drop(drop_list, axis=1)
    for col in df.columns.values:
        if col not in ignore_avg_list:
            # average of all previous gameweeks
            df['avg_' + str(col)] = df[[col]].expanding().mean().shift().fillna(value=0, axis=1)
            # average of last 5 gameweeks
            #df['avg_' + str(col)] = df[[col]].rolling(window=5, min_periods=1).mean().shift().fillna(value=0, axis=1)
            #remove first gameweek since we don't have data about the previous week
            #df = df.iloc[1:]

            if col not in ignore_drop:
                df = df.drop(col, axis=1)
    result.append(df)
    
# merging all the dataframes together
df = pd.concat(result)

df.to_csv("players2019_2020_expanding_shift.csv")
```




Upload the csv file with all player data from 2019-2020 season:

In [ ]:
from google.colab import files
uploaded = files.upload()



Saving players2020_2021_recent_expanding_shift_names.csv to players2020_2021_recent_expanding_shift_names.csv


Load data into dataframe and split:


In [ ]:
import pandas as pd
import io
from sklearn.model_selection import train_test_split


df = pd.read_csv('players2019_2020_expanding_shift.csv', index_col=0)
# df = pd.read_csv(io.StringIO(uploaded['players2019_2020_rolling_5_shift.csv'].decode('utf-8')), index_col=0)
most_recent_gameweek = pd.read_csv('players2020_2021_recent_expanding_shift_names.csv', index_col=0)

features = df.drop('total_points', axis=1)
target = df[['total_points']]


train_features,test_features,train_target,test_target = train_test_split(features,target)
df.head()


,opponent_team,total_points,was_home,avg_assists,last5_assists,avg_bonus,last5_bonus,avg_bps,last5_bps,avg_clean_sheets,last5_clean_sheets,avg_creativity,last5_creativity,avg_goals_conceded,last5_goals_conceded,avg_goals_scored,last5_goals_scored,avg_ict_index,last5_ict_index,avg_influence,last5_influence,avg_minutes,last5_minutes,avg_own_goals,last5_own_goals,avg_penalties_missed,last5_penalties_missed,avg_penalties_saved,last5_penalties_saved,avg_red_cards,last5_red_cards,avg_saves,last5_saves,avg_team_a_score,last5_team_a_score,avg_team_h_score,last5_team_h_score,avg_threat,last5_threat,avg_total_points,last5_total_points,avg_value,last5_value,avg_yellow_cards,last5_yellow_cards
0,11,1,False,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0
1,5,1,True,0.0,0.0,0.0,0.0,1.00,1.00,0.0,0.0,0.100000,0.100000,1.000000,1.000000,0.0,0.0,0.000000,0.000000,0.200000,0.200000,24.000000,24.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,4.000000,4.000000,0.00,0.00,1.0,1.0,45.0,45.0,0.0,0.0
2,13,1,False,0.0,0.0,0.0,0.0,1.00,1.00,0.0,0.0,0.200000,0.200000,1.000000,1.000000,0.0,0.0,1.100000,1.100000,0.600000,0.600000,15.000000,15.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.500000,0.500000,2.500000,2.500000,10.50,10.50,1.0,1.0,45.0,45.0,0.0,0.0
3,6,1,False,0.0,0.0,0.0,0.0,1.00,1.00,0.0,0.0,1.733333,1.733333,0.666667,0.666667,0.0,0.0,1.566667,1.566667,1.066667,1.066667,16.666667,16.666667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.333333,0.333333,1.666667,1.666667,13.00,13.00,1.0,1.0,45.0,45.0,0.0,0.0
4,17,13,True,0.0,0.0,0.0,0.0,1.25,1.25,0.0,0.0,1.450000,1.450000,0.750000,0.750000,0.0,0.0,1.200000,1.200000,0.850000,0.850000,18.750000,18.750000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.250000,0.250000,1.750000,1.750000,9.75,9.75,1.0,1.0,45.0,45.0,0.0,0.0


Cross validate Elasticnet alphas:

In [ ]:
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_squared_error

alphas = [i/100 for i in range(1,101)]
mses = []
for alpha in alphas:
  lr_elastic = ElasticNet(random_state=0, alpha=alpha, l1_ratio=0.5)
  lr_elastic.fit(train_features, train_target)
  test_predictions = lr_elastic.predict(test_features)
  mses.append((alpha, mean_squared_error(test_target.values.flatten(), test_predictions)))

smallest = min(mses, key = lambda t: t[1])
print("Smallest MSE: " + str(smallest[1]) + ", with alpha value: " + str(smallest[0]))

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2455.758283074734, tolerance: 10.362622256458904
  positive)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 313.2756903478585, tolerance: 10.362622256458904
  positive)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 26.979075774375815, tolerance: 10.362622256458904
  positive)


Smallest MSE: 5.020476267869243, with alpha value: 0.01


Let's try Elasticnet with best alpha:

In [ ]:
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_squared_error

alpha = smallest[1]

lr_elastic = ElasticNet(random_state=0, alpha=alpha)

lr_elastic.fit(train_features, train_target)

test_predictions = lr_elastic.predict(test_features)

results_df = pd.DataFrame({'Predicted Points': test_predictions,'Actual Points':test_target.values.flatten()})

results_df.head(20)



,Predicted Points,Actual Points
0,1.483976,2
1,2.977047,1
2,0.393058,0
3,1.601755,1
4,1.158609,0
5,0.390706,0
6,2.863179,11
7,0.872545,1
8,3.113709,0
9,0.371547,0


Predictions of lates gameweek

In [ ]:
most_recent_gameweek["predicted"] = lr_elastic.predict(most_recent_gameweek.drop(["name","total_points"], axis=1))
most_recent_gameweek = most_recent_gameweek[["name","total_points", "predicted"]]
have_points = most_recent_gameweek[most_recent_gameweek["total_points"] > 0]
have_points.sample(15)


,name,total_points,predicted
7,Christian_Benteke_132,1,0.475622
7,Che_Adams_377,7,3.028222
7,Yves_Bissouma_76,2,2.559260
7,Hugo_Lloris_383,6,2.984849
6,Robbie_Brady_95,3,0.619851
7,Kyle_Edwards_421,1,0.825013
7,Pascal_Groß_57,3,1.236312
7,James_McCarthy_131,1,1.346430
7,Giovani_Lo Celso_403,1,1.150420
7,Mason_Mount_120,2,2.246730


Let's try some basic neural network with keras (was_home gives error, probably because it's the only non numerical value) :

In [ ]:
import tensorflow as tf

no_features_columns = len(train_features.keys())
no_target_columns = 1

model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(32,activation='relu',input_shape=[None, 43]),
    tf.keras.layers.Dense(16,activation='relu'),
    tf.keras.layers.Dense(16,activation='relu'),
    tf.keras.layers.Dense(no_target_columns)
])

model.compile(optimizer=tf.keras.optimizers.RMSprop(),
             loss='mean_squared_error',
             metrics=['mean_squared_error'])

history = model.fit(train_features.drop("was_home", axis=1),train_target,epochs=10)



Epoch 1/10
528/528 [==============================] - 1s 1ms/step - loss: 5.4328 - mean_squared_error: 5.4328
Epoch 2/10
528/528 [==============================] - 1s 1ms/step - loss: 4.7293 - mean_squared_error: 4.7293
Epoch 3/10
528/528 [==============================] - 1s 984us/step - loss: 4.7061 - mean_squared_error: 4.7061
Epoch 4/10
528/528 [==============================] - 1s 970us/step - loss: 4.6848 - mean_squared_error: 4.6848
Epoch 5/10
528/528 [==============================] - 1s 1ms/step - loss: 4.6488 - mean_squared_error: 4.6488
Epoch 6/10
528/528 [==============================] - 1s 992us/step - loss: 4.6567 - mean_squared_error: 4.6567
Epoch 7/10
528/528 [==============================] - 0s 938us/step - loss: 4.6525 - mean_squared_error: 4.6525
Epoch 8/10
528/528 [==============================] - 1s 986us/step - loss: 4.6491 - mean_squared_error: 4.6491
Epoch 9/10
528/528 [==============================] - 1s 968us/step - loss: 4.6418 - mean_squared_error: 4.641

In [ ]:
test_predictions = model.predict(test_features.drop('was_home', axis=1)).flatten()
results_df = pd.DataFrame({'Predicted Points': test_predictions,'Actual Points':test_target.values.flatten()})
scores = model.evaluate(test_features.drop('was_home', axis=1), test_target)
print(f"MSE is: {scores[1]}")

176/176 [==============================] - 0s 796us/step - loss: 4.9630 - mean_squared_error: 4.9630
MSE is: 4.963029384613037


Let's try gradient boosting: 

In [ ]:
import xgboost as xgb


# XGBoosts wants data to be wrapped into special formats
dtrain = xgb.DMatrix(train_features, label=train_target)
dtest = xgb.DMatrix(test_features, label=test_target)

# most meaningful parameters
param_list = [("objective", "multi:softmax"), ("eval_metric", "merror")]

# Number of trees
n_rounds = 600

# if nothing seems to improve for 50 iterations - stop
early_stopping = 50

# train for training and test for ... validation!    
eval_list = [(dtrain, 'train'), (dtest, 'eval')]

# 1,2,3.. go!
xgb_r = xgb.XGBRegressor(objective ='reg:squarederror', n_estimators = 10, seed = 123) 

# Fitting the model 
xgb_r.fit(train_features, train_target) 
  
# Predict the model 
test_predictions = xgb_r.predict(test_features) 
  
# RMSE Computation 
mse = mean_squared_error(test_target.values.flatten(), test_predictions)
print("MSE : % f" %(mse)) 

MSE :  5.345591
